In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
[V1]
* Resolution: Resized to 512x512 from 768x768
* Extract cell masks and create individual cell images (512x512)
* No random crop
* No TTA
* Update normalization mean and std with 2021 training and test sets

[V2]
* Use INTER_AREA for resize

[V3]
* Refactor to use less memory

[V4]
* Run batch cell segmentator from script

[V5]
* Add data augmentation

[V6]
* Faster HPA-Cell-Segmentatior (resize to 512x512)

[V7]
* 5 folds ensemble with 1 seed (avg prob.)

[V8]
* 5 folds ensemble with 1 seed (max prob.)

Note: HPA-Cell-Segmentatior assume that all input images are of the same shape!
"""

kernel_mode = False
debug = False

import sys
if kernel_mode:
    sys.path.insert(0, "../input/hpa-bestfitting-solution/src")
    sys.path.insert(0, "../input/hpa-cell-segmentation")

In [3]:
# !ls ../input/hpa-bestfitting-solution/

In [4]:
# !cp ../input/hpa-bestfitting-solution/densenet121-a639ec97.pth .
# !ls -la

In [5]:
# !pip install -q "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
# !pip install -q "../input/hpapytorchzoozip/pytorch_zoo-master"

In [6]:
import sys
import argparse
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import time
import random
import math
import pickle
from pickle import dump, load
import glob
import time
import collections

import torch
import torch.optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler
from torch.nn import DataParallel
import torch.nn.functional as F
from torch.autograd import Variable

from config.config import *
from utils.common_util import *
from networks.imageclsnet import init_network
from datasets.protein_dataset import ProteinDataset
from utils.augment_util import *
from datasets.tool import *

import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics.functional import classification

from pycocotools import _mask as coco_mask
import typing as t
import base64
import zlib

import cv2
from PIL import Image
import imagehash

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

import gc
gc.enable()

rand_seed = 1120

print(f"PyTorch Version: {torch.__version__}")
print(f"PyTorch Lightning Version: {pl.__version__}")

run on 2fb1688be1a5
PyTorch Version: 1.6.0+cu101
PyTorch Lightning Version: 1.1.1


In [7]:
if kernel_mode:
    dataset_folder = "/kaggle/input/hpa-single-cell-image-classification"
    bestfitting_folder = "/kaggle/input/hpa-bestfitting-solution"
    test_image_folder = f"{dataset_folder}/test/"
    cell_mask_folder = "/kaggle/working/test_cell_masks"
    NUC_MODEL = "/kaggle/input/hpa-cell-segmentation/dpn_unet_nuclei_v1.pth"
    CELL_MODEL = "/kaggle/input/hpa-cell-segmentation/dpn_unet_cell_3ch_v1.pth"
else:
    dataset_folder = "/workspace/Kaggle/HPA/hpa_2020"
    bestfitting_folder = "/workspace/Github/HPA-competition-solutions/bestfitting"
    test_image_folder = f"{dataset_folder}/test/"
    cell_mask_folder = f"{dataset_folder}/test_cell_masks"
    NUC_MODEL = "/workspace/Github/HPA-Cell-Segmentation/dpn_unet_nuclei_v1.pth"
    CELL_MODEL = "/workspace/Github/HPA-Cell-Segmentation/dpn_unet_cell_3ch_v1.pth"

model_folder = "external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds"

# image_size = 2048
# image_size = 768
crop_size = 512

batch_size = 512 if kernel_mode else 256
if debug:
    batch_size = 4
# batch_size = 6 if kernel_mode else 4
# batch_size = 4
num_workers = 2 if kernel_mode else 3

# scale_factor = 1.0
# scale_factor = 0.1
scale_factor = 0.25
confidence_threshold = 0.5

In [8]:
old_classes = {
    0: 'Nucleoplasm',
    1: 'Nuclear membrane',
    2: 'Nucleoli',
    3: 'Nucleoli fibrillar center',
    4: 'Nuclear speckles',
    5: 'Nuclear bodies',
    6: 'Endoplasmic reticulum',
    7: 'Golgi apparatus',
    8: 'Peroxisomes',
    9: 'Endosomes',
    10: 'Lysosomes',
    11: 'Intermediate filaments',
    12: 'Actin filaments',
    13: 'Focal adhesion sites',
    14: 'Microtubules',
    15: 'Microtubule ends',
    16: 'Cytokinetic bridge',
    17: 'Mitotic spindle',
    18: 'Microtubule organizing center',
    19: 'Centrosome',
    20: 'Lipid droplets',
    21: 'Plasma membrane',
    22: 'Cell junctions',
    23: 'Mitochondria',
    24: 'Aggresome',
    25: 'Cytosol',
    26: 'Cytoplasmic bodies',
    27: 'Rods & rings'
}
old_class_indices = {v: k for k, v in old_classes.items()}

# All label names in the public HPA and their corresponding index.
all_locations = dict({
    "Nucleoplasm": 0,
    "Nuclear membrane": 1,
    "Nucleoli": 2,
    "Nucleoli fibrillar center": 3,
    "Nuclear speckles": 4,
    "Nuclear bodies": 5,
    "Endoplasmic reticulum": 6,
    "Golgi apparatus": 7,
    "Intermediate filaments": 8,
    "Actin filaments": 9,
    "Focal adhesion sites": 9,
    "Microtubules": 10,
    "Mitotic spindle": 11,
    "Centrosome": 12,
    "Centriolar satellite": 12,
    "Plasma membrane": 13,
    "Cell Junctions": 13,
    "Mitochondria": 14,
    "Aggresome": 15,
    "Cytosol": 16,
    "Vesicles": 17,
    "Peroxisomes": 17,
    "Endosomes": 17,
    "Lysosomes": 17,
    "Lipid droplets": 17,
    "Cytoplasmic bodies": 17,
    "Rods & rings": 18,
    # markpeng
    "No staining": 18,
})

old_class_mappings = {}
for i, (k, v) in enumerate(old_class_indices.items()):
    if k in all_locations:
        old_class_mappings[v] = all_locations[k]
    else:
        # No staining
        old_class_mappings[v] = 18
assert len(old_class_mappings.values()) == len(old_classes.values())
print(old_class_mappings)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 17, 9: 17, 10: 17, 11: 8, 12: 9, 13: 9, 14: 10, 15: 18, 16: 18, 17: 11, 18: 18, 19: 12, 20: 17, 21: 13, 22: 18, 23: 14, 24: 15, 25: 16, 26: 17, 27: 18}


In [9]:
!ls {dataset_folder}

inference	       test		test_tfrecords	train.csv
sample_submission.csv  test_cell_masks	train		train_tfrecords


In [10]:
train_df = pd.read_csv(f"{dataset_folder}/train.csv")
submit_df = pd.read_csv(f"{dataset_folder}/sample_submission.csv")

In [11]:
print(train_df.shape)
train_df.head()

(21806, 2)


,ID,Label
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0


In [12]:
print(submit_df.shape, submit_df.ImageWidth.min(), submit_df.ImageWidth.max())
submit_df.head()

(559, 4) 1728 3072


,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
1,004a270d-34a2-4d60-bbe4-365fca868193,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
2,00537262-883c-4b37-a3a1-a4931b6faea5,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
3,00c9a1c9-2f06-476f-8b0d-6d01032874a2,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==


In [13]:
colors = ["red", "green", "blue", "yellow"]

test_ids = submit_df["ID"].values.tolist()
print(len(test_ids))

# Estimated number of private test images (RGBY): 2236 x 2.3 ~= 5143 (for 9 hours we have 6.2 secs per image)
# Estimated number of private test images: 559 x 2.3 ~= 1286 (for 9 hours we have 25.2 secs per image)

559


## Utility Functions

In [14]:
# Reference: https://www.kaggle.com/dschettler8845/hpa-cellwise-classification-inference/notebook
def binary_mask_to_ascii(mask, mask_val=1):
    """Converts a binary mask into OID challenge encoding ascii text."""
    mask = np.where(mask == mask_val, 1, 0).astype(np.bool)

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
            f"encode_binary_mask expects a binary mask, received dtype == {mask.dtype}"
        )

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            f"encode_binary_mask expects a 2d mask, received shape == {mask.shape}"
        )

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode()


def rle_encoding(img, mask_val=1):
    """
    Turns our masks into RLE encoding to easily store them
    and feed them into models later on
    https://en.wikipedia.org/wiki/Run-length_encoding
    
    Args:
        img (np.array): Segmentation array
        mask_val (int): Which value to use to create the RLE
        
    Returns:
        RLE string
    
    """
    dots = np.where(img.T.flatten() == mask_val)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b

    return ' '.join([str(x) for x in run_lengths])


def rle_to_mask(rle_string, height, width):
    """ Convert RLE sttring into a binary mask 
    
    Args:
        rle_string (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array of the binary segmentation mask for a given cell
    """
    rows, cols = height, width
    rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
    rle_pairs = np.array(rle_numbers).reshape(-1, 2)
    img = np.zeros(rows * cols, dtype=np.uint8)
    for index, length in rle_pairs:
        index -= 1
        img[index:index + length] = 255
    img = img.reshape(cols, rows)
    img = img.T
    return img


def create_segmentation_maps(list_of_image_lists, segmentator, batch_size=8):
    """ Function to generate segmentation maps using CellSegmentator tool 
    
    Args:
        list_of_image_lists (list of lists):
            - [[micro-tubules(red)], [endoplasmic-reticulum(yellow)], [nucleus(blue)]]
        batch_size (int): Batch size to use in generating the segmentation masks
        
    Returns:
        List of lists containing RLEs for all the cells in all images
    """

    all_mask_rles = {}
    for i in tqdm(range(0, len(list_of_image_lists[0]), batch_size),
                  total=len(list_of_image_lists[0]) // batch_size):

        # Get batch of images
        sub_images = [
            img_channel_list[i:i + batch_size]
            for img_channel_list in list_of_image_lists
        ]  # 0.000001 seconds

        # Do segmentation
        cell_segmentations = segmentator.pred_cells(sub_images)
        nuc_segmentations = segmentator.pred_nuclei(sub_images[2])

        # post-processing
        for j, path in enumerate(sub_images[0]):
            img_id = path.replace("_red.png", "").rsplit("/", 1)[1]
            nuc_mask, cell_mask = label_cell(nuc_segmentations[j],
                                             cell_segmentations[j])
            new_name = os.path.basename(path).replace('red', 'mask')
            all_mask_rles[img_id] = [
                rle_encoding(cell_mask, mask_val=k)
                for k in range(1,
                               np.max(cell_mask) + 1)
            ]
    return all_mask_rles


def get_img_list(img_dir, return_ids=False, sub_n=None):
    """ Get image list in the format expected by the CellSegmentator tool """
    if sub_n is None:
        sub_n = len(glob(img_dir + '/' + f'*_red.png'))
    if return_ids:
        images = [
            sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n]
            for c in ["red", "yellow", "blue"]
        ]
        return [
            x.replace("_red.png", "").rsplit("/", 1)[1] for x in images[0]
        ], images
    else:
        return [
            sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n]
            for c in ["red", "yellow", "blue"]
        ]


def get_contour_bbox_from_rle(
    rle,
    width,
    height,
    return_mask=True,
):
    """ Get bbox of contour as `xmin ymin xmax ymax`
    
    Args:
        rle (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array for a cell bounding box coordinates
    """
    mask = rle_to_mask(rle, height, width).copy()
    cnts = grab_contours(
        cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE))
    x, y, w, h = cv2.boundingRect(cnts[0])

    if return_mask:
        return (x, y, x + w, y + h), mask
    else:
        return (x, y, x + w, y + h)


def get_contour_bbox_from_raw(raw_mask):
    """ Get bbox of contour as `xmin ymin xmax ymax`
    
    Args:
        raw_mask (nparray): Numpy array containing segmentation mask information
    
    Returns:
        Numpy array for a cell bounding box coordinates
    """
    cnts = grab_contours(
        cv2.findContours(raw_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE))
    xywhs = [cv2.boundingRect(cnt) for cnt in cnts]
    xys = [(xywh[0], xywh[1], xywh[0] + xywh[2], xywh[1] + xywh[3])
           for xywh in xywhs]
    return sorted(xys, key=lambda x: (x[1], x[0]))


def pad_to_square(a):
    """ Pad an array `a` evenly until it is a square """
    if a.shape[1] > a.shape[0]:  # pad height
        n_to_add = a.shape[1] - a.shape[0]
        top_pad = n_to_add // 2
        bottom_pad = n_to_add - top_pad
        a = np.pad(a, [(top_pad, bottom_pad), (0, 0), (0, 0)], mode='constant')

    elif a.shape[0] > a.shape[1]:  # pad width
        n_to_add = a.shape[0] - a.shape[1]
        left_pad = n_to_add // 2
        right_pad = n_to_add - left_pad
        a = np.pad(a, [(0, 0), (left_pad, right_pad), (0, 0)], mode='constant')
    else:
        pass
    return a


def cut_out_cells(rgby,
                  rles,
                  resize_to=(256, 256),
                  square_off=True,
                  return_masks=False,
                  from_raw=True):
    """ Cut out the cells as padded square images 
    
    Args:
        rgby (np.array): 4 Channel image to be cut into tiles
        rles (list of RLE strings): List of run length encoding containing 
            segmentation mask information
        resize_to (tuple of ints, optional): The square dimension to resize the image to
        square_off (bool, optional): Whether to pad the image to a square or not
        
    Returns:
        list of square arrays representing squared off cell images
    """
    w, h = rgby.shape[:2]
    contour_bboxes = [
        get_contour_bbox(rle, w, h, return_mask=return_masks) for rle in rles
    ]
    if return_masks:
        masks = [x[-1] for x in contour_bboxes]
        contour_bboxes = [x[:-1] for x in contour_bboxes]

    arrs = [
        rgby[bbox[1]:bbox[3], bbox[0]:bbox[2], ...] for bbox in contour_bboxes
    ]
    if square_off:
        arrs = [pad_to_square(arr) for arr in arrs]

    if resize_to is not None:
        arrs = [
            cv2.resize(pad_to_square(arr).astype(np.float32),
                       resize_to,
                       interpolation=cv2.INTER_CUBIC) \
            for arr in arrs
        ]
    if return_masks:
        return arrs, masks
    else:
        return arrs


def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(
            ("Contours tuple must have length 2 or 3, "
             "otherwise OpenCV changed their cv2.findContours return "
             "signature yet again. Refer to OpenCV's documentation "
             "in that case"))

    # return the actual contours array
    return cnts

In [15]:
# https://www.kaggle.com/c/human-protein-atlas-image-classification/discussion/72534
def generate_hash(img_dir,
                  colors,
                  dataset='train',
                  imread_func=None,
                  is_update=False):
    meta = meta.copy()
    hash_maps = {}
    for color in colors:
        hash_maps[color] = []
        for idx in tqdm(range(len(meta)), desc='train %s' % color):
            img = imread_func(img_dir, meta.iloc[idx][ID], color)
            hash = imagehash.phash(img)
            hash_maps[color].append(hash)

    for color in colors:
        meta[color] = hash_maps[color]

    return meta


def calc_hash(params):
    color, threshold, base_test_hash1, base_test_hash2, test_ids1, test_ids2 = params

    test_hash1 = base_test_hash1.reshape(1, -1)  # 1*m

    test_idxes_list1 = []
    test_idxes_list2 = []
    hash_list = []

    step = 5
    for test_idx in tqdm(range(0, len(base_test_hash2), step), desc=color):
        test_hash2 = base_test_hash2[test_idx:test_idx + step].reshape(
            -1, 1)  # n*1
        hash = test_hash2 - test_hash1  # n*m
        test_idxes2, test_idxes1 = np.where(hash <= threshold)
        hash = hash[test_idxes2, test_idxes1]

        test_idxes2 = test_idxes2 + test_idx

        test_idxes_list1.extend(test_idxes1.tolist())
        test_idxes_list2.extend(test_idxes2.tolist())
        hash_list.extend(hash.tolist())

    df = pd.DataFrame({
        'Test1': test_ids1[test_idxes_list1],
        'Test2': test_ids2[test_idxes_list2],
        'Sim%s' % color[:1].upper(): hash_list
    })
    df = df[df['Test1'] != df['Test2']]
    return df

## Preprocessing

### Extract Cell Segmentations as Numpy Files

In [16]:
%%writefile hpa_cell_segment.py

kernel_mode = False

from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import cv2
import gc

import sys
if kernel_mode:
    sys.path.insert(0, "../input/hpa-bestfitting-solution/src")
    sys.path.insert(0, "../input/hpa-cell-segmentation")

from hpacellseg.cellsegmentator import *

target_image_size = 512

IMAGE_SIZES = [1728, 2048, 3072, 4096]
if kernel_mode:
    dataset_folder = "/kaggle/input/hpa-single-cell-image-classification"
    img_dir = f"{dataset_folder}/test"
    output_folder = "/kaggle/working/test_cell_masks"
    NUC_MODEL = "/kaggle/input/hpa-cell-segmentation/dpn_unet_nuclei_v1.pth"
    CELL_MODEL = "/kaggle/input/hpa-cell-segmentation/dpn_unet_cell_3ch_v1.pth"
    BATCH_SIZE = {1728: 24, 2048: 22, 3072: 12, 4096: 12}
else:
    dataset_folder = "/workspace/Kaggle/HPA/hpa_2020"
    img_dir = f"{dataset_folder}/test"
    output_folder = f"{dataset_folder}/test_cell_masks"
    NUC_MODEL = "/workspace/Github/HPA-Cell-Segmentation/dpn_unet_nuclei_v1.pth"
    CELL_MODEL = "/workspace/Github/HPA-Cell-Segmentation/dpn_unet_cell_3ch_v1.pth"
    #     BATCH_SIZE = {1728: 24, 2048: 24, 3072: 12, 4096: 12}
    BATCH_SIZE = {1728: 20, 2048: 18, 3072: 8, 4096: 8}

os.makedirs(output_folder, exist_ok=True)

submit_df = pd.read_csv(f"{dataset_folder}/sample_submission.csv")

predict_df_1728 = submit_df[submit_df.ImageWidth == IMAGE_SIZES[0]]
predict_df_2048 = submit_df[submit_df.ImageWidth == IMAGE_SIZES[1]]
predict_df_3072 = submit_df[submit_df.ImageWidth == IMAGE_SIZES[2]]
predict_df_4096 = submit_df[submit_df.ImageWidth == IMAGE_SIZES[3]]

predict_ids_1728 = predict_df_1728.ID.to_list()
predict_ids_2048 = predict_df_2048.ID.to_list()
predict_ids_3072 = predict_df_3072.ID.to_list()
predict_ids_4096 = predict_df_4096.ID.to_list()


class CellSegmentator(object):
    """Uses pretrained DPN-Unet models to segment cells from images."""
    def __init__(
        self,
        nuclei_model="../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth",
        cell_model="../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth",
        scale_factor=1.0,
        device="cuda",
        padding=False,
        multi_channel_model=True,
    ):
        """Class for segmenting nuclei and whole cells from confocal microscopy images.
        It takes lists of images and returns the raw output from the
        specified segmentation model. Models can be automatically
        downloaded if they are not already available on the system.
        When working with images from the Huan Protein Cell atlas, the
        outputs from this class' methods are well combined with the
        label functions in the utils module.
        Note that for cell segmentation, there are two possible models
        available. One that works with 2 channeled images and one that
        takes 3 channels.
        Keyword arguments:
        nuclei_model -- A loaded torch nuclei segmentation model or the
                        path to a file which contains such a model.
                        If the argument is a path that points to a non-existant file,
                        a pretrained nuclei_model is going to get downloaded to the
                        specified path (default: './nuclei_model.pth').
        cell_model -- A loaded torch cell segmentation model or the
                      path to a file which contains such a model.
                      The cell_model argument can be None if only nuclei
                      are to be segmented (default: './cell_model.pth').
        scale_factor -- How much to scale images before they are fed to
                        segmentation models. Segmentations will be scaled back
                        up by 1/scale_factor to match the original image
                        (default: 0.25).
        device -- The device on which to run the models.
                  This should either be 'cpu' or 'cuda' or pointed cuda
                  device like 'cuda:0' (default: 'cuda').
        padding -- Whether to add padding to the images before feeding the
                   images to the network. (default: False).
        multi_channel_model -- Control whether to use the 3-channel cell model or not.
                               If True, use the 3-channel model, otherwise use the
                               2-channel version (default: True).
        """
        if device != "cuda" and device != "cpu" and "cuda" not in device:
            raise ValueError(f"{device} is not a valid device (cuda/cpu)")
        if device != "cpu":
            try:
                assert torch.cuda.is_available()
            except AssertionError:
                print("No GPU found, using CPU.", file=sys.stderr)
                device = "cpu"
        self.device = device

        if isinstance(nuclei_model, str):
            if not os.path.exists(nuclei_model):
                print(
                    f"Could not find {nuclei_model}. Downloading it now",
                    file=sys.stderr,
                )
                download_with_url(NUCLEI_MODEL_URL, nuclei_model)
            nuclei_model = torch.load(nuclei_model,
                                      map_location=torch.device(self.device))
        if isinstance(nuclei_model, torch.nn.DataParallel) and device == "cpu":
            nuclei_model = nuclei_model.module

        self.nuclei_model = nuclei_model.to(self.device).eval()

        self.multi_channel_model = multi_channel_model
        if isinstance(cell_model, str):
            if not os.path.exists(cell_model):
                print(f"Could not find {cell_model}. Downloading it now",
                      file=sys.stderr)
                if self.multi_channel_model:
                    download_with_url(MULTI_CHANNEL_CELL_MODEL_URL, cell_model)
                else:
                    download_with_url(TWO_CHANNEL_CELL_MODEL_URL, cell_model)
            cell_model = torch.load(cell_model,
                                    map_location=torch.device(self.device))
        self.cell_model = cell_model.to(self.device).eval()
        self.scale_factor = scale_factor
        self.padding = padding

    def _image_conversion(self, images):
        """Convert/Format images to RGB image arrays list for cell predictions.
        Intended for internal use only.
        Keyword arguments:
        images -- list of lists of image paths/arrays. It should following the
                 pattern if with er channel input,
                 [
                     [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                     [er_path0/image_array0, er_path1/image_array1, ...],
                     [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                 ]
                 or if without er input,
                 [
                     [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                     None,
                     [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                 ]
        """
        microtubule_imgs, er_imgs, nuclei_imgs = images
        if self.multi_channel_model:
            if not isinstance(er_imgs, list):
                raise ValueError(
                    "Please speicify the image path(s) for er channels!")
        else:
            if not er_imgs is None:
                raise ValueError(
                    "second channel should be None for two channel model predition!"
                )

        if not isinstance(microtubule_imgs, list):
            raise ValueError("The microtubule images should be a list")
        if not isinstance(nuclei_imgs, list):
            raise ValueError("The microtubule images should be a list")

        if er_imgs:
            if not len(microtubule_imgs) == len(er_imgs) == len(nuclei_imgs):
                raise ValueError(
                    "The lists of images needs to be the same length")
        else:
            if not len(microtubule_imgs) == len(nuclei_imgs):
                raise ValueError(
                    "The lists of images needs to be the same length")

        if not all(isinstance(item, np.ndarray) for item in microtubule_imgs):
            microtubule_imgs = [
                os.path.expanduser(item)
                for _, item in enumerate(microtubule_imgs)
            ]
            nuclei_imgs = [
                os.path.expanduser(item) for _, item in enumerate(nuclei_imgs)
            ]

            microtubule_imgs = list(
                map(lambda item: imageio.imread(item), microtubule_imgs))
            nuclei_imgs = list(
                map(lambda item: imageio.imread(item), nuclei_imgs))
            if er_imgs:
                er_imgs = [
                    os.path.expanduser(item) for _, item in enumerate(er_imgs)
                ]
                er_imgs = list(map(lambda item: imageio.imread(item), er_imgs))

        if not er_imgs:
            er_imgs = [
                np.zeros(item.shape, dtype=item.dtype)
                for _, item in enumerate(microtubule_imgs)
            ]
        cell_imgs = list(
            map(
                lambda item: np.dstack((item[0], item[1], item[2])),
                list(zip(microtubule_imgs, er_imgs, nuclei_imgs)),
            ))

        return cell_imgs

    def pred_nuclei(self, images, bs=24):
        """Predict the nuclei segmentation.
        Keyword arguments:
        images -- A list of image arrays or a list of paths to images.
                  If as a list of image arrays, the images could be 2d images
                  of nuclei data array only, or must have the nuclei data in
                  the blue channel; If as a list of file paths, the images
                  could be RGB image files or gray scale nuclei image file
                  paths.
        Returns:
        predictions -- A list of predictions of nuclei segmentation for each nuclei image.
        """
        def _preprocess(image):
            if isinstance(image, str):
                image = imageio.imread(image)
            self.target_shape = image.shape
            if len(image.shape) == 2:
                image = np.dstack((image, image, image))
            image = transform.rescale(image,
                                      self.scale_factor,
                                      multichannel=True)
            nuc_image = np.dstack((image[..., 2], image[..., 2], image[...,
                                                                       2]))
            if self.padding:
                rows, cols = nuc_image.shape[:2]
                self.scaled_shape = rows, cols
                nuc_image = cv2.copyMakeBorder(
                    nuc_image,
                    32,
                    (32 - rows % 32),
                    32,
                    (32 - cols % 32),
                    cv2.BORDER_REFLECT,
                )
            nuc_image = nuc_image.transpose([2, 0, 1])
            return nuc_image

        def _segment_helper(imgs):
            with torch.no_grad():
                mean = torch.as_tensor(NORMALIZE["mean"], device=self.device)
                std = torch.as_tensor(NORMALIZE["std"], device=self.device)
                imgs = torch.tensor(imgs).float()
                imgs = imgs.to(self.device)
                imgs = imgs.sub_(mean[:, None, None]).div_(std[:, None, None])

                imgs = self.nuclei_model(imgs)
                imgs = F.softmax(imgs, dim=1)
                return imgs

        preprocessed_imgs = list(map(_preprocess, images))
        predictions = []
        for i in range(0, len(preprocessed_imgs), bs):
            start = i
            end = min(len(preprocessed_imgs), i + bs)
            x = preprocessed_imgs[start:end]
            pred = _segment_helper(x).cpu().numpy()
            predictions.append(pred)
        predictions = list(np.concatenate(predictions, axis=0))
        predictions = map(util.img_as_ubyte, predictions)
        predictions = list(map(self._restore_scaling_padding, predictions))
        return predictions

    def _restore_scaling_padding(self, n_prediction):
        """Restore an image from scaling and padding.
        This method is intended for internal use.
        It takes the output from the nuclei model as input.
        """
        n_prediction = n_prediction.transpose([1, 2, 0])
        if self.padding:
            n_prediction = n_prediction[32:32 + self.scaled_shape[0],
                                        32:32 + self.scaled_shape[1], ...]
        if not self.scale_factor == 1:
            n_prediction[..., 0] = 0
            n_prediction = cv2.resize(
                n_prediction,
                (self.target_shape[0], self.target_shape[1]),
                # interpolation=cv2.INTER_AREA,
                interpolation=cv2.INTER_NEAREST,
            )
        return n_prediction

    def pred_cells(self, images, precombined=False, bs=24):
        """Predict the cell segmentation for a list of images.
        Keyword arguments:
        images -- list of lists of image paths/arrays. It should following the
                  pattern if with er channel input,
                  [
                      [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                      [er_path0/image_array0, er_path1/image_array1, ...],
                      [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                  ]
                  or if without er input,
                  [
                      [microtubule_path0/image_array0, microtubule_path1/image_array1, ...],
                      None,
                      [nuclei_path0/image_array0, nuclei_path1/image_array1, ...]
                  ]
                  The ER channel is required when multichannel is True
                  and required to be None when multichannel is False.
                  The images needs to be of the same size.
        precombined -- If precombined is True, the list of images is instead supposed to be
                       a list of RGB numpy arrays (default: False).
        Returns:
        predictions -- a list of predictions of cell segmentations.
        """
        def _preprocess(image):
            self.target_shape = image.shape
            if not len(image.shape) == 3:
                raise ValueError("image should has 3 channels")
            cell_image = transform.rescale(image,
                                           self.scale_factor,
                                           multichannel=True)
            if self.padding:
                rows, cols = cell_image.shape[:2]
                self.scaled_shape = rows, cols
                cell_image = cv2.copyMakeBorder(
                    cell_image,
                    32,
                    (32 - rows % 32),
                    32,
                    (32 - cols % 32),
                    cv2.BORDER_REFLECT,
                )
            cell_image = cell_image.transpose([2, 0, 1])
            return cell_image

        def _segment_helper(imgs):
            with torch.no_grad():
                mean = torch.as_tensor(NORMALIZE["mean"], device=self.device)
                std = torch.as_tensor(NORMALIZE["std"], device=self.device)
                imgs = torch.tensor(imgs).float()
                imgs = imgs.to(self.device)
                imgs = imgs.sub_(mean[:, None, None]).div_(std[:, None, None])

                imgs = self.cell_model(imgs)
                imgs = F.softmax(imgs, dim=1)
                return imgs

        if not precombined:
            images = self._image_conversion(images)
        preprocessed_imgs = list(map(_preprocess, images))
        predictions = []
        for i in range(0, len(preprocessed_imgs), bs):
            start = i
            end = min(len(preprocessed_imgs), i + bs)
            x = preprocessed_imgs[start:end]
            pred = _segment_helper(x).cpu().numpy()
            predictions.append(pred)
        predictions = list(np.concatenate(predictions, axis=0))
        predictions = map(self._restore_scaling_padding, predictions)
        predictions = list(map(util.img_as_ubyte, predictions))

        return predictions


import os.path
import urllib
import zipfile

import numpy as np
import scipy.ndimage as ndi
from skimage import filters, measure, segmentation
from skimage.morphology import (binary_erosion, closing, disk,
                                remove_small_holes, remove_small_objects)

HIGH_THRESHOLD = 0.4
LOW_THRESHOLD = HIGH_THRESHOLD - 0.25


def download_with_url(url_string, file_path, unzip=False):
    """Download file with a link."""
    with urllib.request.urlopen(url_string) as response, open(
            file_path, "wb") as out_file:
        data = response.read()  # a `bytes` object
        out_file.write(data)

    if unzip:
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(os.path.dirname(file_path))


def __fill_holes(image):
    """Fill_holes for labelled image, with a unique number."""
    boundaries = segmentation.find_boundaries(image)
    image = np.multiply(image, np.invert(boundaries))
    image = ndi.binary_fill_holes(image > 0)
    image = ndi.label(image)[0]
    return image


def label_cell(nuclei_pred, cell_pred):
    """Label the cells and the nuclei.
    Keyword arguments:
    nuclei_pred -- a 3D numpy array of a prediction from a nuclei image.
    cell_pred -- a 3D numpy array of a prediction from a cell image.
    Returns:
    A tuple containing:
    nuclei-label -- A nuclei mask data array.
    cell-label  -- A cell mask data array.
    0's in the data arrays indicate background while a continous
    strech of a specific number indicates the area for a specific
    cell.
    The same value in cell mask and nuclei mask refers to the identical cell.
    NOTE: The nuclei labeling from this function will be sligthly
    different from the values in :func:`label_nuclei` as this version
    will use information from the cell-predictions to make better
    estimates.
    """
    def __wsh(
        mask_img,
        threshold,
        border_img,
        seeds,
        threshold_adjustment=0.35,
        small_object_size_cutoff=10,
    ):
        img_copy = np.copy(mask_img)
        m = seeds * border_img  # * dt
        img_copy[m <= threshold + threshold_adjustment] = 0
        img_copy[m > threshold + threshold_adjustment] = 1
        img_copy = img_copy.astype(np.bool)
        img_copy = remove_small_objects(
            img_copy, small_object_size_cutoff).astype(np.uint8)

        mask_img[mask_img <= threshold] = 0
        mask_img[mask_img > threshold] = 1
        mask_img = mask_img.astype(np.bool)
        mask_img = remove_small_holes(mask_img, 63)
        mask_img = remove_small_objects(mask_img, 1).astype(np.uint8)
        markers = ndi.label(img_copy, output=np.uint32)[0]
        labeled_array = segmentation.watershed(mask_img,
                                               markers,
                                               mask=mask_img,
                                               watershed_line=True)
        return labeled_array

    nuclei_label = __wsh(
        nuclei_pred[..., 2] / 255.0,
        0.4,
        1 - (nuclei_pred[..., 1] + cell_pred[..., 1]) / 255.0 > 0.05,
        nuclei_pred[..., 2] / 255,
        threshold_adjustment=-0.25,
        small_object_size_cutoff=32,
    )

    # for hpa_image, to remove the small pseduo nuclei
    nuclei_label = remove_small_objects(nuclei_label, 157)
    nuclei_label = measure.label(nuclei_label)
    # this is to remove the cell borders' signal from cell mask.
    # could use np.logical_and with some revision, to replace this func.
    # Tuned for segmentation hpa images
    threshold_value = max(
        0.22,
        filters.threshold_otsu(cell_pred[..., 2] / 255) * 0.5)
    # exclude the green area first
    cell_region = np.multiply(
        cell_pred[..., 2] / 255 > threshold_value,
        np.invert(np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8)),
    )
    sk = np.asarray(cell_region, dtype=np.int8)
    distance = np.clip(cell_pred[..., 2], 255 * threshold_value, cell_pred[...,
                                                                           2])
    cell_label = segmentation.watershed(-distance, nuclei_label, mask=sk)
    cell_label = remove_small_objects(cell_label, 344).astype(np.uint8)
    selem = disk(2)
    cell_label = closing(cell_label, selem)
    cell_label = __fill_holes(cell_label)
    # this part is to use green channel, and extend cell label to green channel
    # benefit is to exclude cells clear on border but without nucleus
    sk = np.asarray(
        np.add(
            np.asarray(cell_label > 0, dtype=np.int8),
            np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8),
        ) > 0,
        dtype=np.int8,
    )
    cell_label = segmentation.watershed(-distance, cell_label, mask=sk)
    cell_label = __fill_holes(cell_label)
    cell_label = np.asarray(cell_label > 0, dtype=np.uint8)
    cell_label = measure.label(cell_label)
    cell_label = remove_small_objects(cell_label, 344)
    cell_label = measure.label(cell_label)
    cell_label = np.asarray(cell_label, dtype=np.uint16)
    nuclei_label = np.multiply(cell_label > 0, nuclei_label) > 0
    nuclei_label = measure.label(nuclei_label)
    nuclei_label = remove_small_objects(nuclei_label, 157)
    nuclei_label = np.multiply(cell_label, nuclei_label > 0)

    return nuclei_label, cell_label


segmentator = CellSegmentator(NUC_MODEL, CELL_MODEL, padding=True)
# segmentator = CellSegmentator(
#     NUC_MODEL,
#     CELL_MODEL,
#     scale_factor=0.25,
#     device="cuda",
#     padding=True,
#     multi_channel_model=True,
# )


def get_segment_mask(gray, rgb, bs=24):
    nuc_segmentations = segmentator.pred_nuclei(gray, bs=bs)  # blue
    cell_segmentations = segmentator.pred_cells(rgb, precombined=True, bs=bs)
    batch_cell_masks = [
        label_cell(nuc_seg, cell_seg)[1].astype(np.uint8)
        for nuc_seg, cell_seg in zip(nuc_segmentations, cell_segmentations)
    ]
    return batch_cell_masks


def load_images(image_ids, root=img_dir):
    gray = []
    rgb = []
    for ID in tqdm(image_ids, total=len(image_ids)):
        r = os.path.join(root, f'{ID}_red.png')
        y = os.path.join(root, f'{ID}_yellow.png')
        b = os.path.join(root, f'{ID}_blue.png')
        r = cv2.imread(r, 0)
        y = cv2.imread(y, 0)
        b = cv2.imread(b, 0)
        gray_image = cv2.resize(b, (target_image_size, target_image_size))
        rgb_image = cv2.resize(np.stack((r, y, b), axis=2),
                               (target_image_size, target_image_size))
        gray.append(gray_image)
        rgb.append(rgb_image)
    return gray, rgb


for size_idx, submission_ids in tqdm(enumerate(
    [predict_ids_1728, predict_ids_2048, predict_ids_3072, predict_ids_4096]),
                                     total=4):
    size = IMAGE_SIZES[size_idx]
    if submission_ids == []:
        print(f"\n...SKIPPING SIZE {size} AS THERE ARE NO IMAGE IDS ...\n")
        continue
    else:
        print(f"\n...WORKING ON IMAGE IDS FOR SIZE {size} ...\n")
    for i in tqdm(range(0, len(submission_ids), BATCH_SIZE[size]),
                  total=int(np.ceil(len(submission_ids) / BATCH_SIZE[size]))):

        r, y, b = [], [], []
        image_ids = submission_ids[i:(i + BATCH_SIZE[size])]
        gray, rgb = load_images(image_ids)
        batch_cell_masks = get_segment_mask(gray, rgb, bs=BATCH_SIZE[size])

        torch.cuda.empty_cache()
        gc.collect()

        for index, img_id in enumerate(image_ids):
            np.save(os.path.join(output_folder, f'{img_id}_cell_mask.npy'),
                    batch_cell_masks[index])

Overwriting hpa_cell_segment.py


In [17]:
# %%time
# !python hpa_cell_segment.py

In [18]:
# Wall time: 4min 45s

### Calculate Mean and Std From Training and Test Sets

In [19]:
total_mean = [0.081018, 0.052349, 0.054012, 0.08106] # rgby
total_std = [0.133235, 0.08948, 0.143813, 0.130265]

## Load Pretrained Model from Bestfitting

In [20]:
parser = argparse.ArgumentParser(description='PyTorch Protein Classification')
parser.add_argument('--out_dir', type=str, help='destination where predicted result should be saved')
parser.add_argument('--gpu_id', default='0', type=str, help='gpu id used for predicting (default: 0)')
parser.add_argument('--arch', default='class_densenet121_dropout', type=str,
                    help='model architecture (default: class_densenet121_dropout)')
parser.add_argument('--num_classes', default=28, type=int, help='number of classes (default: 28)')
parser.add_argument('--in_channels', default=4, type=int, help='in channels (default: 4)')
parser.add_argument('--img_size', default=768, type=int, help='image size (default: 768)')
parser.add_argument('--crop_size', default=512, type=int, help='crop size (default: 512)')
parser.add_argument('--batch_size', default=32, type=int, help='train mini-batch size (default: 32)')
parser.add_argument('--workers', default=3, type=int, help='number of data loading workers (default: 3)')
parser.add_argument('--fold', default=0, type=int, help='index of fold (default: 0)')
parser.add_argument('--augment', default='default', type=str, help='test augmentation (default: default)')
parser.add_argument('--seed', default=100, type=int, help='random seed (default: 100)')
parser.add_argument('--seeds', default=None, type=str, help='predict seed')
parser.add_argument('--predict_epoch', default=None, type=int, help='number epoch to predict')

_StoreAction(option_strings=['--predict_epoch'], dest='predict_epoch', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help='number epoch to predict', metavar=None)

In [21]:
args = parser.parse_args([
    "--arch",
    "class_densenet121_dropout",
    #     "--img_size", str(image_size),
    "--crop_size",
    str(crop_size),
])
args

Namespace(arch='class_densenet121_dropout', augment='default', batch_size=32, crop_size=512, fold=0, gpu_id='0', img_size=768, in_channels=4, num_classes=28, out_dir=None, predict_epoch=None, seed=100, seeds=None, workers=3)

In [22]:
def load_model(network_path, args, print_model=False):
    # setting up the visible GPU
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id

    model_params = {}
    model_params['architecture'] = args.arch
    model_params['num_classes'] = args.num_classes
    model_params['in_channels'] = args.in_channels
    model_params['pretrained_path'] = f"{bestfitting_folder}"
    model = init_network(model_params)

    checkpoint = torch.load(network_path)
    model.load_state_dict(checkpoint['state_dict'])

    # moving network to gpu and eval mode
    # model = DataParallel(model)
    model.cuda()
    model.eval()

    if print_model:
        print(model)

    return model

## Inference

In [23]:
def read_crop_img(img):
    random_crop_size = int(np.random.uniform(self.crop_size, self.img_size))
    x = int(np.random.uniform(0, self.img_size - random_crop_size))
    y = int(np.random.uniform(0, self.img_size - random_crop_size))
    crop_img = img[x:x + random_crop_size, y:y + random_crop_size]
    return crop_img


def read_rgby(
    img_dir,
    img_id,
    random_crop=False,
):
    suffix = '.png'
    colors = ['red', 'green', 'blue', 'yellow']

    flags = cv2.IMREAD_GRAYSCALE
    rgby_img = [
        cv2.imread(opj(img_dir, img_id + '_' + color + suffix), flags)
        for color in colors
    ]
    rgby_img = np.stack(rgby_img, axis=-1)
    if random_crop and crop_size > 0:
        rgby_img = read_crop_img(rgby_img)

    return rgby_img

In [24]:
def process_image(img_id, img_dir):
    if img_id not in global_cache:
        rgby_img = read_rgby(img_dir, img_id)
        if rgby_img[0] is None:
            print(self.img_dir, img_id)

        h, w = rgby_img.shape[:2]

        if crop_size > 0:
            if crop_size != h or crop_size != w:
                resized_rgby_img = cv2.resize(rgby_img, (crop_size, crop_size),
                                              interpolation=cv2.INTER_LINEAR)

        full_mask = np.load(f"{cell_mask_folder}/{img_id}_cell_mask.npy")

        #         full_mask = cv2.resize(full_mask, (crop_size, crop_size),
        #                                interpolation=cv2.INTER_NEAREST)

        cell_masks = [
            rle_encoding(full_mask, mask_val=k)
            for k in range(1,
                           np.max(full_mask) + 1)
        ]
        if len(cell_masks) == 0:
            print(f"No cell masks found for {img_id}")

        resized_rgby_img = resized_rgby_img / 255.0

        global_cache[img_id] = (resized_rgby_img, cell_masks)
        return resized_rgby_img, cell_masks
    else:
        # print(f"Cache hit for {img_id}!")
        return global_cache[img_id]

In [25]:
def collate_cells_fn(x):
    images = []
    # For each full image, extract cell images
    image = x[0]
    masks = x[1]
    transform = x[2]

    for rle_string in masks:
        cell_mask = rle_to_mask(rle_string, crop_size, crop_size)
        # Important: set 255 to 1
        cell_mask[cell_mask > 0] = 1

        cell_image = np.copy(image)
        #         cell_mask = np.tile(cell_mask, reps=(4, 1, 1)).transpose(1, 2, 0)
        #         cell_image *= cell_mask
        for i in range(4):
            cell_image[..., i] = cell_image[..., i] * cell_mask

        if transform is not None:
            cell_image = transform(cell_image)

        cell_image = image_to_tensor(cell_image)
        images.append(cell_image.unsqueeze(0))

    images = torch.cat(images)
    return images, masks

In [26]:
def predict(df, model, generate_meta=False):
    all_probs = []
    all_meta = {}

    global_processed = 0
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        id = row["ID"]
        width = row["ImageWidth"]
        height = row["ImageHeight"]

        # Data augmentation
        aug_probs = None
        for augment in augment_list:
            transform = eval(f"augment_{augment}")
            images, masks = process_image(id, test_image_folder)
            images, masks = collate_cells_fn((images, masks, transform))

            cell_probs = []
            processed_count = 0
            with torch.no_grad():
                for batch_i in range(0, len(masks), batch_size):
                    batch_images = images[batch_i:batch_i + batch_size, ...]
                    # batch_images = Variable(batch_images.cuda(), volatile=True)
                    outputs = model(batch_images.cuda())
                    logits = outputs

                    probs = F.sigmoid(logits).data
                    probs = probs.detach().cpu().numpy().tolist()
                    cell_probs += probs

                    processed_count += len(probs)

            cell_probs = np.array(cell_probs).reshape(processed_count, -1)

            # Take maximum probability
            if aug_probs is None:
                aug_probs = cell_probs
            else:
                aug_probs = np.max(np.stack([aug_probs, cell_probs], axis=-1),
                                   axis=-1)

        all_probs.append(aug_probs)

        if generate_meta:
            masks = np.array(masks)

            if masks.shape[0] > 0:
                # Generate RLE string for each cell mask
                # https://www.kaggle.com/dschettler8845/hpa-cellwise-classification-inference/notebook?scriptVersionId=55714434
                submit_strings = []
                for i in range(masks.shape[0]):
                    mask = masks[i]
                    mask = rle_to_mask(mask, crop_size, crop_size)
                    # Important: set 255 to 1
                    mask[mask > 0] = 1

                    # Important: resize to orignal resolution to submit correct mask RLE string
                    # https://www.kaggle.com/linshokaku/faster-hpa-cell-segmentation/comments#1251082
                    mask = cv2.resize(mask, (width, height),
                                      interpolation=cv2.INTER_NEAREST)

                    rle_string = binary_mask_to_ascii(mask, mask_val=1)
                    submit_strings.append(rle_string)

                if len(submit_strings) > 0:
                    all_meta[id] = submit_strings
                else:
                    all_meta[id] = []
            else:
                all_meta[id] = []

        del images, masks, aug_probs, cell_probs, batch_images
        gc.collect()

        if debug and global_processed == batch_size - 1:
            break

        global_processed += 1

    gc.collect()

    if generate_meta:
        return all_probs, all_meta
    else:
        return all_probs

In [ ]:
%%time

final_probs = []
final_meta = {}

augment_list = [
    'default', 'flipud', 'fliplr', 'transpose', 'flipud_lr',
    'flipud_transpose', 'fliplr_transpose', 'flipud_lr_transpose'
]
# seeds = [0, 1, 2, 3]
seeds = [0]
kfolds = 5
# fold = 0

cache_size = batch_size if debug else 250
global_cache = {}
batch_rounds = 0
for batch_i in range(0, submit_df.shape[0], cache_size):
    print(f"[Batch Processing {batch_rounds}]")
    sub_df = submit_df.iloc[batch_i:batch_i + cache_size, :].copy()
    print(sub_df.shape)

    batch_probs = [0] * sub_df.shape[0]
    for fold in range(kfolds):
        print(f"[Fold {fold}]")
        network_path = f"{bestfitting_folder}/{model_folder}/fold{fold}/final.pth"
        model = load_model(network_path, args)

        for i, seed in enumerate(seeds):
            print(f"Inferencing with seed {rand_seed+1000*seed} ......")
            seed_everything(rand_seed + 1000 * seed)

            if fold == 0 and i == 0:
                seed_probs, meta = predict(sub_df, model, generate_meta=True)
                print(len(seed_probs), len(batch_probs), sub_df.shape[0])
                for j in range(len(seed_probs)):
                    batch_probs[j] = seed_probs[j] / (len(seeds) * kfolds)
                final_meta.update(meta)
            else:
                seed_probs = predict(sub_df, model)
                print(len(seed_probs), len(batch_probs), sub_df.shape[0])
                for j in range(len(seed_probs)):
                    batch_probs[j] += seed_probs[j] / (len(seeds) * kfolds)

        del model
        torch.cuda.empty_cache()
        gc.collect()

    if batch_probs is not None:
        final_probs.extend(batch_probs)

    if debug:
        break

    # Reset cache
    del global_cache
    gc.collect()
    global_cache = {}
    batch_rounds += 1

[Batch Processing 0]
(4, 4)
[Fold 0]
>> Using pre-trained model.


  0%|          | 0/4 [00:00<?, ?it/s]

Inferencing with seed 1120 ......
[[3.62206995e-01 6.96077719e-02 3.42795521e-01 1.60231054e-01
  8.20998922e-02 2.51484305e-01 1.02459334e-01 1.92332745e-01
  2.28999322e-03 3.53545113e-03 6.28912169e-03 7.71953166e-02
  7.59235099e-02 5.83368912e-02 1.29011972e-02 3.71978711e-03
  1.38330460e-01 5.83171472e-02 2.53898978e-01 2.45704949e-01
  2.10448522e-02 2.19062492e-01 1.62080884e-01 1.62196666e-01
  9.07187685e-02 4.04913962e-01 1.22781083e-01 1.08080953e-02]
 [2.01792195e-01 1.17822349e-01 4.89691019e-01 1.90024972e-01
  5.43352738e-02 1.63085893e-01 2.51752079e-01 2.28718311e-01
  4.81699593e-03 1.08769918e-02 2.91119013e-02 6.31397218e-02
  5.09570390e-02 2.06030942e-02 1.39095969e-02 3.78557667e-03
  9.30549279e-02 2.95455977e-02 2.01911405e-01 1.47595048e-01
  7.94452950e-02 1.37688652e-01 1.13683447e-01 1.91155523e-01
  1.61625683e-01 5.51715493e-01 1.08014360e-01 3.91528048e-02]
 [5.20580888e-01 1.61313519e-01 2.93175489e-01 1.86735824e-01
  6.67452142e-02 2.29509726e-01 8.

[[3.57700586e-01 7.59760514e-02 2.28237346e-01 1.25934079e-01
  9.47658196e-02 2.29719147e-01 9.03181732e-02 1.50201574e-01
  2.88130133e-03 4.33140341e-03 6.47036824e-03 8.85031745e-02
  7.39921182e-02 6.29944578e-02 2.74885632e-02 9.65327956e-03
  1.94592491e-01 1.10098489e-01 3.50608975e-01 3.52669805e-01
  1.36118187e-02 1.85313001e-01 1.30724013e-01 1.96562648e-01
  8.21591839e-02 4.33756918e-01 9.48744714e-02 2.07045879e-02]
 [2.12598681e-01 9.06938314e-02 4.66409147e-01 2.29278430e-01
  4.10217382e-02 1.71139374e-01 2.49421507e-01 1.75520703e-01
  6.29008980e-03 1.01367068e-02 2.10126583e-02 9.14758220e-02
  1.07526407e-01 3.64414342e-02 2.23758928e-02 8.48207064e-03
  1.21816784e-01 4.69629727e-02 2.17052996e-01 1.99180037e-01
  7.17288032e-02 1.68956205e-01 1.32841721e-01 1.67875782e-01
  2.63699085e-01 5.95530629e-01 1.23803467e-01 4.02182713e-02]
 [5.46522558e-01 1.78533643e-01 2.11854234e-01 1.72716811e-01
  6.40940741e-02 2.48138949e-01 6.13579117e-02 1.08868442e-01
  9.58

[[3.15064102e-01 6.89197704e-02 3.76763165e-01 1.81221083e-01
  8.69971439e-02 2.21644640e-01 1.04228705e-01 1.59771189e-01
  2.78674555e-03 3.73134250e-03 5.79594728e-03 6.88114390e-02
  6.60226196e-02 5.45490831e-02 1.34618403e-02 4.33492754e-03
  1.32286042e-01 6.41810969e-02 2.47346967e-01 2.63034225e-01
  1.88546367e-02 2.13720277e-01 1.68691173e-01 1.55509591e-01
  7.82041252e-02 4.17915553e-01 1.19962089e-01 1.14511279e-02]
 [2.30667204e-01 1.09104104e-01 5.24812698e-01 2.28325740e-01
  4.42033634e-02 1.59402952e-01 2.32607812e-01 3.29648852e-01
  6.35042042e-03 8.55808426e-03 2.60099694e-02 6.17035627e-02
  4.37441990e-02 2.29756366e-02 1.37925297e-02 3.09777469e-03
  9.05328766e-02 2.84709912e-02 1.94054052e-01 1.42576307e-01
  5.70547245e-02 1.30789950e-01 1.33016512e-01 2.11701259e-01
  1.79306388e-01 5.01044154e-01 7.06581622e-02 1.92416646e-02]
 [5.43004155e-01 1.75301552e-01 3.02973360e-01 1.35827318e-01
  7.55878165e-02 2.30709568e-01 7.26494417e-02 1.04942486e-01
  1.70

...

[[2.83387959e-01 5.38975745e-02 3.25549543e-01 2.46934503e-01
  7.95736834e-02 2.03488961e-01 1.01127677e-01 1.84049547e-01
  3.32286046e-03 2.78560072e-03 3.98967229e-03 8.31988007e-02
  7.09724948e-02 3.60022411e-02 1.30235150e-01 7.20118126e-03
  2.65929729e-01 1.09664448e-01 1.63352311e-01 1.76216334e-01
  1.78598855e-02 1.92335695e-01 1.29943088e-01 2.12291434e-01
  4.59857881e-02 4.98995423e-01 9.23368633e-02 6.50261482e-03]
 [1.32447511e-01 3.28995436e-02 5.18784404e-01 2.33004615e-01
  2.58103106e-02 8.86388645e-02 1.85785502e-01 2.03297108e-01
  7.59722723e-04 3.64556606e-03 6.21210132e-03 6.09457567e-02
  3.68720070e-02 2.37358287e-02 5.10921329e-02 4.21825127e-04
  1.34170353e-01 3.49139646e-02 1.26587436e-01 1.88570902e-01
  4.39332798e-02 1.51829630e-01 8.47413018e-02 2.03140751e-01
  5.27377166e-02 5.67286432e-01 9.70029160e-02 7.19756959e-03]
 [4.04791981e-01 1.47323117e-01 2.70132780e-01 2.55756617e-01
  1.12809338e-01 2.07095891e-01 9.76716578e-02 1.31160766e-01
  1.54

[[3.46356690e-01 8.25362355e-02 2.67223120e-01 1.74186483e-01
  9.25517306e-02 2.51023769e-01 1.02357581e-01 1.71180472e-01
  2.40206323e-03 3.56401922e-03 5.09654125e-03 7.89341033e-02
  8.10169801e-02 3.75530384e-02 9.65382084e-02 7.40967039e-03
  2.43648291e-01 9.99138132e-02 1.95088029e-01 2.24171534e-01
  1.12853283e-02 1.86545089e-01 1.26527503e-01 1.62508681e-01
  5.42925596e-02 4.93118078e-01 1.05154306e-01 6.88092550e-03]
 [1.38519049e-01 4.83016483e-02 5.79762161e-01 2.68442571e-01
  4.23956588e-02 1.29955098e-01 1.02066725e-01 2.87505299e-01
  4.29410441e-03 1.97165087e-03 4.34410386e-03 9.23804045e-02
  4.26502936e-02 2.33686473e-02 8.11811537e-02 2.10830593e-03
  1.55978143e-01 5.31725176e-02 1.26958042e-01 1.59658343e-01
  1.66318156e-02 1.69289827e-01 1.02329865e-01 2.86636531e-01
  4.49080057e-02 5.18671572e-01 4.59343605e-02 7.13734329e-03]
 [3.15221846e-01 1.49136394e-01 3.64337802e-01 2.74908185e-01
  8.23133737e-02 1.89529985e-01 9.42793190e-02 1.18186869e-01
  8.57

 25%|██▌       | 1/4 [00:02<00:06,  2.07s/it]

[[3.60574901e-01 7.21443668e-02 2.70008028e-01 2.06766933e-01
  8.97703469e-02 2.37493321e-01 9.56341922e-02 1.43956974e-01
  3.08747217e-03 2.03838921e-03 2.66893487e-03 8.40465724e-02
  9.45347920e-02 4.88715619e-02 1.17374144e-01 9.35104769e-03
  2.56203681e-01 1.01026140e-01 1.55626819e-01 1.84679180e-01
  1.07852230e-02 2.11430535e-01 1.50336877e-01 1.92062199e-01
  4.85462360e-02 4.74793732e-01 8.98802727e-02 5.04428893e-03]
 [1.54398784e-01 3.80484760e-02 5.91464400e-01 2.37170830e-01
  3.94613966e-02 1.08139373e-01 1.32107928e-01 1.77120119e-01
  1.01336150e-03 1.45852554e-03 2.47530849e-03 9.05437618e-02
  3.24784778e-02 2.20020954e-02 4.79491539e-02 6.62171806e-04
  1.10200249e-01 4.44083400e-02 1.20474987e-01 1.75459638e-01
  2.19939575e-02 1.88018650e-01 9.19092745e-02 1.87650010e-01
  4.39947173e-02 5.91942430e-01 5.53468987e-02 5.32132667e-03]
 [3.64415854e-01 1.74923062e-01 3.20133656e-01 2.41217315e-01
  9.80202854e-02 1.95192620e-01 9.43268165e-02 1.28473252e-01
  1.69

[[2.55931646e-01 3.35288979e-02 1.79527804e-01 1.18949153e-01
  5.74818812e-02 1.49303734e-01 8.64197388e-02 1.61312237e-01
  4.90212115e-04 1.80763495e-03 2.05007056e-03 8.86101052e-02
  7.41186291e-02 4.72800955e-02 1.73602104e-01 4.81603807e-03
  3.33019793e-01 3.12578708e-01 3.02313268e-01 2.37674057e-01
  6.83272164e-03 2.15104014e-01 1.13528743e-01 1.34802133e-01
  2.01280024e-02 5.47976196e-01 1.03095859e-01 8.67875014e-03]
 [3.16526741e-01 6.08417541e-02 1.58181280e-01 1.40719473e-01
  6.13360889e-02 1.76249728e-01 1.36329472e-01 2.25100726e-01
  1.49484864e-03 3.87124927e-03 4.87211719e-03 1.03711471e-01
  8.73878598e-02 4.10260707e-02 1.05373412e-01 2.46523973e-03
  2.17470035e-01 1.42179593e-01 2.07075328e-01 2.12273300e-01
  1.06200762e-02 2.22670570e-01 1.12515844e-01 1.81261390e-01
  4.32848185e-02 5.19114017e-01 1.07115969e-01 8.31341557e-03]
 [2.00257391e-01 1.08562939e-01 2.09327444e-01 7.90842772e-02
  7.85702392e-02 8.93710032e-02 1.76973388e-01 1.87836066e-01
  7.32

In [ ]:
len(final_probs)

In [ ]:
all_predictions = []
for index, row in tqdm(submit_df.iterrows(), total=submit_df.shape[0]):
    id = row["ID"]
    width = row["ImageWidth"]
    height = row["ImageHeight"]

    cell_probs = final_probs[index]
    rle_strings = final_meta[id]

    new_preds = np.zeros((cell_probs.shape[0], 19))
    for i in range(cell_probs.shape[0]):
        for j in range(28):
            new_class_i = old_class_mappings[j]
            # Take maximum prob.
            if cell_probs[i, j] > new_preds[i, new_class_i]:
                new_preds[i, new_class_i] = cell_probs[i, j]

    submit_strings = []
    for i in range(new_preds.shape[0]):
        confidence = new_preds[i, ...]
        rle_string = rle_strings[i]
        for l in range(19):
            submit_strings.append(f"{l} {confidence[l]:.6f} {rle_string}")

    if len(submit_strings) > 0:
        all_predictions.append(" ".join(submit_strings))
    else:
        all_predictions.append("")

    if debug and index == batch_size - 1:
        break

In [ ]:
len(all_predictions)

In [ ]:
if debug:
    submit_df.iloc[:batch_size, :]["PredictionString"] = all_predictions
else:
    submit_df["PredictionString"] = all_predictions
submit_df

In [ ]:
submit_df.to_csv("submission.csv", index=False)

In [ ]:
def prob_to_result(probs, img_ids, th=0.5):
    probs = np.concatenate(probs, axis=0)
    predicted_probs = probs.copy()
    probs[np.arange(len(probs)), np.argmax(probs, axis=1)] = 1

    pred_list = []
    pred_list_new = []
    for line in probs:
        # Map old classes to new ones
        predicted_old_classes = sorted(
            list(set([i for i in np.nonzero(line > th)[0]])))
        predicted_new_classes = sorted(
            list(set([old_class_mappings[i]
                      for i in np.nonzero(line > th)[0]])))
        # print(predicted_classes)
        s = '|'.join([str(i) for i in predicted_old_classes])
        s_new = '|'.join([str(i) for i in predicted_new_classes])
        pred_list.append(s)
        pred_list_new.append(s_new)
    result_df = pd.DataFrame({
        # "ID": img_ids,
        "Predicted": pred_list,
        "Predicted_New": pred_list_new
    })
    return result_df

In [ ]:
result_df = prob_to_result(final_probs, None, th=confidence_threshold)
result_df.to_csv("result_comparison.csv", index=False)
result_df.head()

In [ ]:
result_df.head(100)

In [ ]:
result_df["Predicted_New"].value_counts()

In [ ]:
# test_dataset.release_gpu()
# del model, test_dataset, test_loader
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# !rm densenet121-a639ec97.pth hpa_cell_segment.py
# !rm -rf inference test_cell_masks
# !ls -la

### EOF